<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9D%B4%EC%9A%A9%EB%AF%BC_42000_%EC%A0%9C%EB%A1%9C%EC%83%B7_5%EB%9D%BC%EB%B2%A8%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os
from tqdm.notebook import tqdm

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/프로젝트/final project/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

# df = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0905_clean.csv")
# df.info()

Mounted at /content/drive


'cuda'

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.1 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

# 모델과 토크나이저 초기화
model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
model = AutoModelForSequenceClassification.from_pretrained(model_name).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
result_df = pd.read_csv(f"{DATA_PATH}train_ft_42000_0922.csv")

In [ ]:
result_df

,review,reply,sentiment,score,kiwi_reviews,category
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,1,0.566494,"['맛있', '먹', '묵사발', '시원', '맛있']",['맛']
1,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,고객님께서도 행복하시길 바랄게요,1,0.608408,"['양', '푸짐', '맛있', '청국장', '냄새', '나', '콩', '들', ...","['맛', '양']"
2,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,고객님의 행복한 시간에 저희 매장이 함께 할 수 있어 너무나도 영광입니다! 즐거움이...,1,0.381793,"['냉면', '시키', '미니', '냉면', '합치', '양', '적', '보이',...",['양']
3,맛나게 잘 묵었습니다,"고객님의 즐거운 순간을 함께할 수 있기를 바라며, 기다리고 있겠습니다 더욱 발전된 ...",1,0.549870,"['맛나', '묵']",['맛']
4,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",1,0.778149,"['맛', '양', '만족', '배달', '보내', '만족']","['맛', '양', '배달']"
...,...,...,...,...,...,...
42539,언제나 맛있게 먹었는데 이번에는 음식이 지루하고 별로였어요. 아쉬워요,"닭도리탕의 맛이 매우 좋지 않고, 김치가 물렸다는 점 죄송합니다. 앞으로 더 맛있는...",0,0.827452,"['맛있', '먹', '이번', '음식', '지루하']",['맛']
42540,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...",고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하셨다니 정말 ...,0,0.942769,"['맛', '음식', '촉촉', '식', '먹', '그렇', '다음', '시키', ...","['맛', '서비스']"
42541,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723,"['맛', '없', '배달', '늦', '실망']","['맛', '배달']"
42542,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0,0.894588,"['최악', '맛', '없', '배달', '걸리']","['맛', '배달']"


In [ ]:
result_df["category"].unique()

array(["['맛']", "['맛', '양']", "['양']", "['맛', '양', '배달']", "['맛', '서비스']",
       "['맛', '배달']", "['맛', '양', '가격']", "['배달']",
       "['맛', '서비스', '양', '배달']", "['맛', '서비스', '배달']",
       "['맛', '서비스', '양']", "['맛', '가격']", "['맛', '서비스', '가격']",
       "['서비스', '배달']", "['배달', '맛', '양', '가격', '서비스']",
       "['맛', '서비스', '양', '가격']", "['맛', '배달', '가격']",
       "['맛', '양', '배달', '가격']", "['서비스', '양', '배달']", "['서비스', '양']",
       "['양', '배달']", "['양', '배달', '가격']", "['맛', '서비스', '배달', '가격']",
       "['양', '가격']", "['배달', '가격']", "['서비스', '양', '배달', '가격']",
       "['서비스', '양', '가격']", "['서비스', '배달', '가격']"], dtype=object)

In [ ]:
result_df.rename(columns={
    'review': 'Review',
    'category': 'Label'
}, inplace=True)
result_df = result_df[["Review", "Label"]]

In [ ]:
result_df

,Review,Label
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,['맛']
1,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,"['맛', '양']"
2,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,['양']
3,맛나게 잘 묵었습니다,['맛']
4,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"['맛', '양', '배달']"
...,...,...
42539,언제나 맛있게 먹었는데 이번에는 음식이 지루하고 별로였어요. 아쉬워요,['맛']
42540,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","['맛', '서비스']"
42541,"맛도 없고, 배달도 늦어서 너무 실망했습니다","['맛', '배달']"
42542,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,"['맛', '배달']"


In [ ]:
result_df = result_df.dropna().reset_index(drop=True)

In [ ]:
re_1000 = result_df[:40000]
re_1001 = result_df[40000:42500]

In [ ]:
re_1000

,Review,Label
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,['맛']
1,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,"['맛', '양']"
2,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,['양']
3,맛나게 잘 묵었습니다,['맛']
4,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"['맛', '양', '배달']"
...,...,...
39995,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","['맛', '서비스']"
39996,배달이 너무 느려서 식사 시간이 피해버렸어요. 이 정도면 좀 불편했어요,['배달']
39997,맛도 없고 양도 적었어요. 돈 아깝게 주고 먹은 것 같아요,"['맛', '양']"
39998,음식이 싱거워서 먹을 맛이 없었어요. 다른 식당을 찾아봐야겠어요,['맛']


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
type(re_1000["Label"])

pandas.core.series.Series

In [ ]:
import pandas as pd
import numpy as np

# 레이블을 숫자로 변환하는 딕셔너리
label_to_num = {0: '맛', 1: '양', 2: '가격', 3: '배달', 4: '서비스'}

def string_to_list(label_str):
    if isinstance(label_str, str):
        # 작은 따옴표를 제거하고 split
        return [label.replace("'", "").strip() for label in label_str[1:-1].split(',')]
    return label_str

# labels_to_onehot 함수 수정
def labels_to_onehot(labels, label_to_num):
    onehot = [0]*len(label_to_num)
    for label in labels:
        for key, value in label_to_num.items():
            if label == value:
                onehot[key] = 1
    return onehot

# re_1000 데이터에 대한 문자열 레이블을 리스트로 변환
re_1000['Label_List'] = re_1000['Label'].apply(string_to_list)

# re_1000 데이터에 대한 리스트 형태의 레이블을 one-hot encoding으로 변환
re_1000['OneHot_Labels'] = re_1000['Label_List'].apply(lambda x: labels_to_onehot(x, label_to_num))

# re_1001 데이터에 대한 문자열 레이블을 리스트로 변환
re_1001['Label_List'] = re_1001['Label'].apply(string_to_list)

# re_1001 데이터에 대한 리스트 형태의 레이블을 one-hot encoding으로 변환
re_1001['OneHot_Labels'] = re_1001['Label_List'].apply(lambda x: labels_to_onehot(x, label_to_num))

# re_1000 데이터 프레임의 일부를 출력하여 확인
print(re_1000[['Review', 'Label_List', 'OneHot_Labels']].head())


                                              Review  Label_List  \
0                          맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요         [맛]   
1    양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!      [맛, 양]   
2  냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...         [양]   
3                                        맛나게 잘 묵었습니다         [맛]   
4            맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.  [맛, 양, 배달]   

     OneHot_Labels  
0  [1, 0, 0, 0, 0]  
1  [1, 1, 0, 0, 0]  
2  [0, 1, 0, 0, 0]  
3  [1, 0, 0, 0, 0]  
4  [1, 1, 0, 1, 0]  


<ipython-input-19-8c1553143bad>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  re_1000['Label_List'] = re_1000['Label'].apply(string_to_list)
<ipython-input-19-8c1553143bad>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  re_1000['OneHot_Labels'] = re_1000['Label_List'].apply(lambda x: labels_to_onehot(x, label_to_num))
<ipython-input-19-8c1553143bad>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [ ]:
re_1000[['Review', 'Label_List', 'OneHot_Labels']]

,Review,Label_List,OneHot_Labels
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,[맛],"[1, 0, 0, 0, 0]"
1,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,"[맛, 양]","[1, 1, 0, 0, 0]"
2,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,[양],"[0, 1, 0, 0, 0]"
3,맛나게 잘 묵었습니다,[맛],"[1, 0, 0, 0, 0]"
4,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"[맛, 양, 배달]","[1, 1, 0, 1, 0]"
...,...,...,...
39995,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","[맛, 서비스]","[1, 0, 0, 0, 1]"
39996,배달이 너무 느려서 식사 시간이 피해버렸어요. 이 정도면 좀 불편했어요,[배달],"[0, 0, 0, 1, 0]"
39997,맛도 없고 양도 적었어요. 돈 아깝게 주고 먹은 것 같아요,"[맛, 양]","[1, 1, 0, 0, 0]"
39998,음식이 싱거워서 먹을 맛이 없었어요. 다른 식당을 찾아봐야겠어요,[맛],"[1, 0, 0, 0, 0]"


In [ ]:
re_1001[['Review', 'Label_List', 'OneHot_Labels']]

,Review,Label_List,OneHot_Labels
40000,배달기사 분이 무례하게 대응해서 기분이 상했어요. 다시 시키지 않을 거에요,[배달],"[0, 0, 0, 1, 0]"
40001,음식이 변색되어 있었고 맛도 이상했어요. 좀 더 신선한 음식을 원했는데 아쉬웠어요,[맛],"[1, 0, 0, 0, 0]"
40002,음식이 늦게 도착했고 따뜻하지 않아서 맛도 별로였어요. 좀 더 신경써서 준비해주세요,"[맛, 배달]","[1, 0, 0, 1, 0]"
40003,주문한 음식이 다른 음식으로 바뀌어서 실망했어요. 이런 일이 또 발생하면 안될 것 ...,[맛],"[1, 0, 0, 0, 0]"
40004,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","[맛, 서비스]","[1, 0, 0, 0, 1]"
...,...,...,...
42495,음식이 너무 많이 짜서 입맛에 잘 맞지 않았어요. 별로였어요,[맛],"[1, 0, 0, 0, 0]"
42496,음식이 너무 느끼하고 육즙이 없어서 싱겁게 먹기가 어려웠어요. 별로였어요,[맛],"[1, 0, 0, 0, 0]"
42497,음식이 너무 뜨거워서 먹기가 힘들었어요. 맛도 별로였어요,[맛],"[1, 0, 0, 0, 0]"
42498,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","[맛, 서비스]","[1, 0, 0, 0, 1]"


In [ ]:
import numpy as np
from transformers import AutoTokenizer

# 모델 이름 및 토크나이저 정의
model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# 주어진 데이터프레임에서 텍스트와 라벨을 인코딩하는 함수
def process_data(df, tokenizer):
    texts = df["Review"].tolist()
    labels_encoded = np.array(df["OneHot_Labels"].tolist())

    # 텍스트 인코딩
    encodings = tokenizer(texts, padding=True, truncation=True, max_length=256, return_tensors="pt")
    return encodings, labels_encoded

# 각 데이터셋에 대한 처리
train_encodings, train_labels_encoded = process_data(re_1000, tokenizer)
val_encodings, val_labels_encoded = process_data(re_1001, tokenizer)


In [ ]:
# 첫 5개의 훈련 데이터 및 라벨 출력
for i in range(5):
    print("Review:", re_1000["Review"].iloc[i])
    print("Encoded:", train_encodings['input_ids'][i])
    print("Label:", train_labels_encoded[i])
    print("-" * 50)

# 첫 5개의 검증 데이터 및 라벨 출력
for i in range(5):
    print("Review:", re_1001["Review"].iloc[i])
    print("Encoded:", val_encodings['input_ids'][i])
    print("Label:", val_labels_encoded[i])
    print("-" * 50)


Review: 맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요
Encoded: tensor([     1,    260,  35099,  54198,   2318,  14985,  82509,  76883,    260,
        126646,   3073,   9381,   1248,   6464,   3348, 138603,    260,  35099,
         54198,  38646,      2,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      

In [ ]:
# from torch.utils.data import Dataset, DataLoader

# class CustomDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         # item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item = {key: val[idx].clone().detach().requires_grad_(True) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])

#         return item

#     def __len__(self):
#         return len(self.labels)

# # 데이터를 데이터셋으로 변환
# train_dataset = CustomDataset(train_encodings, train_labels_encoded)
# val_dataset = CustomDataset(val_encodings, val_labels_encoded)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float).to('cuda')  # dtype을 명시적으로 설정
        return item

    def __len__(self):
        return len(self.labels)

# 데이터를 데이터셋으로 변환
train_dataset = CustomDataset(train_encodings, train_labels_encoded)
val_dataset = CustomDataset(val_encodings, val_labels_encoded)

# dataloader 생성

In [ ]:
from torch.utils.data import DataLoader

# DataLoader 설정
batch_size = 16  # 이 값은 메모리 및 학습 설정에 따라 조정해야 할 수 있습니다.

train_dataset = CustomDataset(train_encodings, train_labels_encoded)
val_dataset = CustomDataset(val_encodings, val_labels_encoded)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

from transformers import AutoModelForSequenceClassification
from torch.optim import AdamW
from torch.nn import BCEWithLogitsLoss

# 모델, 손실 함수, 옵티마이저 초기화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, ignore_mismatched_sizes=True).to('cuda')
loss_fn = BCEWithLogitsLoss().to('cuda')
optimizer = AdamW(model.parameters(), lr=1e-5)


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at MoritzLaurer/mDeBERTa-v3-base-mnli-xnli and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tqdm import tqdm
import time

epochs = 5
total_steps = len(train_loader) * epochs
model.train()

for epoch in range(epochs):
    total_loss = 0
    epoch_start_time = time.time()  # 에폭 시작 시간 저장

    # tqdm을 사용하여 학습의 진행 상황을 보여줌
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):

        # 배치 데이터를 CUDA에 할당
        inputs = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')

        # 예측 및 손실 계산
        outputs = model(inputs, attention_mask=attention_mask)
        logits = outputs.logits
        loss = loss_fn(logits, labels.float())

        # 역전파
        loss.backward()

        # 가중치 업데이트
        optimizer.step()

        # 그래디언트 초기화
        optimizer.zero_grad()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    epoch_time_elapsed = time.time() - epoch_start_time  # 에폭 경과 시간 계산

    # 학습 로스와 에폭 경과 시간 출력
    print(f"  평균 학습 loss: {avg_train_loss:.4f}")
    print(f"  에폭 토크 학습: {epoch_time_elapsed:.2f}s")

    model.save_pretrained(f"{DATA_PATH}42000_5라벨_{epoch+1}")
    tokenizer.save_pretrained(f"{DATA_PATH}42000_5라벨_{epoch+1}")
    print(f"  에폭 저장 {epoch+1}")

print("완료")


Epoch 1/5: 100%|██████████| 2500/2500 [38:57<00:00,  1.07it/s]


  평균 학습 loss: 0.0898
  에폭 토크 학습: 2337.30s
  에폭 저장 1


Epoch 2/5: 100%|██████████| 2500/2500 [38:57<00:00,  1.07it/s]


  평균 학습 loss: 0.0236
  에폭 토크 학습: 2337.08s
  에폭 저장 2


Epoch 3/5: 100%|██████████| 2500/2500 [38:56<00:00,  1.07it/s]


  평균 학습 loss: 0.0164
  에폭 토크 학습: 2336.63s
  에폭 저장 3


Epoch 4/5: 100%|██████████| 2500/2500 [38:56<00:00,  1.07it/s]


  평균 학습 loss: 0.0136
  에폭 토크 학습: 2336.91s
  에폭 저장 4


Epoch 5/5: 100%|██████████| 2500/2500 [38:58<00:00,  1.07it/s]


  평균 학습 loss: 0.0111
  에폭 토크 학습: 2338.22s
  에폭 저장 5
완료


In [ ]:
model_zeroshot = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/프로젝트/final project/data/42000_5라벨_5")


In [ ]:
print(model)
print(model.config)


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(251000, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine

In [ ]:
model_zeroshot = model_zeroshot.to('cuda')

model_zeroshot.eval()  # 모델을 평가 모드로 설정

total_loss = 0
correct_predictions = 0
total_predictions = 0

for batch in val_loader:
    with torch.no_grad():  # 기울기 계산을 하지 않음
        inputs = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')

        outputs = model_zeroshot(inputs, attention_mask=attention_mask)
        logits = outputs.logits
        loss = loss_fn(logits, labels.float())
        total_loss += loss.item()

        # 로지스틱 회귀 결과를 확률로 변환
        probs = torch.sigmoid(logits)
        # 확률을 라벨 (0 or 1)로 변환
        preds = (probs > 0.5).long()

        correct_predictions += (preds == labels).sum().item()
        total_predictions += labels.numel()

avg_val_loss = total_loss / len(val_loader)
accuracy = correct_predictions / total_predictions

print(f"Validation Loss: {avg_val_loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")


Validation Loss: 0.0060
Validation Accuracy: 0.9987


In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# 데이터 추출
result_re1001 = result_df["Review"][40000:42000].tolist()

# 1. 데이터 전처리
inputs = tokenizer(result_re1001, truncation=True, padding=True, return_tensors="pt")

# TensorDataset 및 DataLoader 구성
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
data_loader = DataLoader(dataset, batch_size=16, shuffle=False)  # 배치 크기는 메모리에 따라 조정할 수 있습니다.

# 2. 모델 예측
all_preds = []
with torch.no_grad():
    for batch in data_loader:
        input_ids, attention_mask = (item.to('cuda') for item in batch)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.sigmoid(logits)
        preds = (probs > 0.5).long()  # 확률을 라벨 (0 or 1)로 변환
        all_preds.extend(preds.cpu().tolist())

print(all_preds)


[[1, 0, 0, 1, 0], [1, 0, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1], [1, 0, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 1], [1, 0, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 1], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1], [1, 0, 0, 0, 0], [1, 0, 0, 1, 1], [1, 0, 0, 1, 1], [1, 0, 0, 0, 0], [1, 0, 0, 1, 

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset

# 레이블을 숫자로 변환하는 딕셔너리
label_to_num = {0: '맛', 1: '양', 2: '가격', 3: '배달', 4: '서비스'}

# 모델 및 토크나이저 불러오기
# model = AutoModelForSequenceClassification.from_pretrained("rere_30000").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", use_fast=False)


# 데이터 추출
result_re30000 = result_df["Review"][40000:42000].tolist()



# 1. 데이터 전처리
inputs = tokenizer(result_re1001, truncation=True, padding=True, return_tensors="pt")

# TensorDataset 및 DataLoader 구성
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
data_loader = DataLoader(dataset, batch_size=16, shuffle=False)

# 2. 모델 예측
all_preds = []
with torch.no_grad():
    for batch in data_loader:
        input_ids, attention_mask = (item.to('cuda') for item in batch)
        outputs = model_zeroshot(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.sigmoid(logits)
        preds = (probs > 0.5).long()  # 확률을 라벨 (0 or 1)로 변환
        all_preds.extend(preds.cpu().tolist())

# 예측된 원-핫 인코딩된 라벨을 텍스트 라벨로 변환하는 함수
def one_hot_to_labels(one_hot_labels, label_to_num):
    labels_list = []
    for one_hot in one_hot_labels:
        labels = [label_to_num[i] for i, value in enumerate(one_hot) if value == 1]
        labels_list.append(labels)
    return labels_list

# 예측된 원-핫 인코딩된 라벨을 텍스트 라벨로 변환
predicted_label_list = one_hot_to_labels(all_preds, label_to_num)

# re_1001 데이터프레임에 예측된 라벨을 추가
result_df["Predicted_Labels"] = [""] * 40000 + predicted_label_list + [""] * (len(result_df) - 42000)

# 결과 출력
print(result_df[['Review', 'Predicted_Labels']].iloc[40000:42000])
result_df.to_csv(DATA_PATH + "42000_zeroshot_fine.csv", index=False)

                                                  Review Predicted_Labels
40000         배달기사 분이 무례하게 대응해서 기분이 상했어요. 다시 시키지 않을 거에요           [맛, 배달]
40001     음식이 변색되어 있었고 맛도 이상했어요. 좀 더 신선한 음식을 원했는데 아쉬웠어요               [맛]
40002    음식이 늦게 도착했고 따뜻하지 않아서 맛도 별로였어요. 좀 더 신경써서 준비해주세요           [맛, 배달]
40003  주문한 음식이 다른 음식으로 바뀌어서 실망했어요. 이런 일이 또 발생하면 안될 것 ...              [맛]
40004  맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...         [맛, 서비스]
...                                                  ...              ...
41995                              맛도 별로고 배달이 느려서 아쉬웠어요           [맛, 배달]
41996                             배달이 너무 느리고 음식도 맛이 없어요           [맛, 배달]
41997                    배달 시간이 오래 걸리고 음식도 맛이 없어서 실망했어요           [맛, 배달]
41998                   음식이 맛이 없었어요. 다른 가게에서 시킬 걸 그랬어요.               [맛]
41999                  음식이 식어서 맛이 없어졌어요. 다신 시키지 않을 거예요.               [맛]

[2000 rows x 2 columns]


In [ ]:
result_df[40000:42000]

,Review,Label,Predicted_Labels
40000,배달기사 분이 무례하게 대응해서 기분이 상했어요. 다시 시키지 않을 거에요,['배달'],"[맛, 배달]"
40001,음식이 변색되어 있었고 맛도 이상했어요. 좀 더 신선한 음식을 원했는데 아쉬웠어요,['맛'],[맛]
40002,음식이 늦게 도착했고 따뜻하지 않아서 맛도 별로였어요. 좀 더 신경써서 준비해주세요,"['맛', '배달']","[맛, 배달]"
40003,주문한 음식이 다른 음식으로 바뀌어서 실망했어요. 이런 일이 또 발생하면 안될 것 ...,['맛'],[맛]
40004,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","['맛', '서비스']","[맛, 서비스]"
...,...,...,...
41995,맛도 별로고 배달이 느려서 아쉬웠어요,"['맛', '배달']","[맛, 배달]"
41996,배달이 너무 느리고 음식도 맛이 없어요,"['맛', '배달']","[맛, 배달]"
41997,배달 시간이 오래 걸리고 음식도 맛이 없어서 실망했어요,"['맛', '배달']","[맛, 배달]"
41998,음식이 맛이 없었어요. 다른 가게에서 시킬 걸 그랬어요.,['맛'],[맛]


In [ ]:
reviews_to_predict = [
    '맛은 있는데 양은 적어요',
    '가성비가 별로 안좋아요',
    '배 터질것 같아요',
    '이렇게 늦을 거면 안 시켜 먹었죠',
    '가격에 비해 양이 좀 적어요',
    '맛있는데 너무 늦게 왔어요',
    '맛에 비해 너무 비싸요',
    '다시는 안 시킬 것 같아요',
    '또 시켜먹을게요',
    '주문하고 30분 내로 왔어요',
    '혜자스럽네요',
    '창렬스럽네요',
    '이물질이 나왔어요',
    '국밥 국물이 끝내줘요',
    '치킨은 매번 여기서 시켜먹어요',


]


# 제로샷 파이프라인 사용(fine-tuning모델)

In [ ]:
#model_zeroshot

# 제로샷 분류 파이프라인 초기화
classifier = pipeline("zero-shot-classification", model=model_zeroshot, tokenizer="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", device=0)  # GPU 사용 설정

# 분류를 위한 레이블 목록
candidate_labels = ['맛', '양', '가격', '배달', '서비스']

# 전체 리뷰에 대해 제로샷 분류 수행
labels_collected = []
scores_collected = []

for review in tqdm(reviews_to_predict):  # tqdm을 사용해 진행 상태 표시
    output = classifier(review, candidate_labels, multi_label=True)
    labels = [label for label, score in zip(output['labels'], output['scores']) if score > 0.5]
    scores = [round(score, 2) for score in output['scores']]

    labels_collected.append(', '.join(labels))
    scores_collected.append(', '.join([f"{label}: {round(score, 2)}" for label, score in zip(output['labels'], output['scores'])]))

# 결과를 데이터프레임으로 변환
lal = pd.DataFrame({
    'Review': reviews_to_predict,
    'Predicted_Labels': labels_collected,
    'Label_Scores': scores_collected
})

# 결과 출력
lal

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.
 67%|██████▋   | 10/15 [00:01<00:00,  5.98it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 15/15 [00:02<00:00,  6.07it/s]


,Review,Predicted_Labels,Label_Scores
0,맛은 있는데 양은 적어요,,"서비스: 0.23, 가격: 0.0, 배달: 0.0, 맛: 0.0, 양: 0.0"
1,가성비가 별로 안좋아요,서비스,"서비스: 0.78, 배달: 0.15, 양: 0.0, 가격: 0.0, 맛: 0.0"
2,배 터질것 같아요,"서비스, 배달","서비스: 0.6, 배달: 0.58, 양: 0.0, 가격: 0.0, 맛: 0.0"
3,이렇게 늦을 거면 안 시켜 먹었죠,,"서비스: 0.08, 배달: 0.0, 가격: 0.0, 맛: 0.0, 양: 0.0"
4,가격에 비해 양이 좀 적어요,서비스,"서비스: 0.99, 배달: 0.07, 양: 0.06, 가격: 0.06, 맛: 0.0"
5,맛있는데 너무 늦게 왔어요,,"서비스: 0.13, 배달: 0.0, 가격: 0.0, 맛: 0.0, 양: 0.0"
6,맛에 비해 너무 비싸요,서비스,"서비스: 0.57, 배달: 0.0, 맛: 0.0, 가격: 0.0, 양: 0.0"
7,다시는 안 시킬 것 같아요,서비스,"서비스: 0.66, 배달: 0.47, 양: 0.0, 가격: 0.0, 맛: 0.0"
8,또 시켜먹을게요,,"서비스: 0.11, 배달: 0.0, 가격: 0.0, 맛: 0.0, 양: 0.0"
9,주문하고 30분 내로 왔어요,서비스,"서비스: 0.68, 배달: 0.47, 양: 0.0, 가격: 0.0, 맛: 0.0"


# 제로샷 fine-tuning 결과

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# zeroshot_model30000 = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/프로젝트/final project/data/zero_model_epoch_2").to('cuda')
# model_zeroshot
candidate_labels = ['맛', '양', '가격', '배달', '서비스']
# 모델 및 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", use_fast=False)

# 데이터 전처리
inputs = tokenizer(reviews_to_predict, truncation=True, padding=True, return_tensors="pt").to('cuda')

# DataLoader 구성
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
data_loader = DataLoader(dataset, batch_size=16, shuffle=False)

labels_collected = []
scores_collected = []

with torch.no_grad():
    for batch in tqdm(data_loader):
        input_ids, attention_mask = (item.to('cuda') for item in batch)
        outputs = model_zeroshot(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.sigmoid(logits).cpu().numpy()

        for prob in probs:
            labels = [candidate_labels[idx] for idx, val in enumerate(prob) if val > 0.5]
            scores = {candidate_labels[idx]: f"{val:.2f}" for idx, val in enumerate(prob)}

            labels_collected.append(", ".join(labels))
            scores_collected.append(", ".join([f"{k}: {v}" for k, v in scores.items()]))

# 결과를 데이터 프레임으로 저장
result_df = pd.DataFrame({
    'Review': reviews_to_predict,
    'Predicted_Labels': labels_collected,
    'Label_Scores': scores_collected
})

result_df

100%|██████████| 1/1 [00:00<00:00, 13.12it/s]


,Review,Predicted_Labels,Label_Scores
0,맛은 있는데 양은 적어요,"맛, 양","맛: 1.00, 양: 0.99, 가격: 0.00, 배달: 0.00, 서비스: 0.00"
1,가성비가 별로 안좋아요,"맛, 가격","맛: 1.00, 양: 0.00, 가격: 1.00, 배달: 0.00, 서비스: 0.01"
2,배 터질것 같아요,맛,"맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 0.00, 서비스: 0.00"
3,이렇게 늦을 거면 안 시켜 먹었죠,맛,"맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 0.00, 서비스: 0.00"
4,가격에 비해 양이 좀 적어요,"양, 가격","맛: 0.25, 양: 0.99, 가격: 1.00, 배달: 0.00, 서비스: 0.01"
5,맛있는데 너무 늦게 왔어요,맛,"맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 0.00, 서비스: 0.00"
6,맛에 비해 너무 비싸요,"맛, 가격","맛: 1.00, 양: 0.00, 가격: 1.00, 배달: 0.00, 서비스: 0.01"
7,다시는 안 시킬 것 같아요,맛,"맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 0.00, 서비스: 0.00"
8,또 시켜먹을게요,맛,"맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 0.00, 서비스: 0.00"
9,주문하고 30분 내로 왔어요,맛,"맛: 1.00, 양: 0.00, 가격: 0.00, 배달: 0.00, 서비스: 0.00"


# 제로샷 "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli" 모델 사용 결과

In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from tqdm import tqdm

# 모델과 토크나이저 초기화
model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
model = AutoModelForSequenceClassification.from_pretrained(model_name).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 제로샷 분류 파이프라인 초기화
classifier = pipeline("zero-shot-classification", model=model, tokenizer=tokenizer, device=0)  # GPU 사용 설정

# 분류를 위한 레이블 목록
candidate_labels = ['맛', '양', '가격', '배달', '서비스']

# 전체 리뷰에 대해 제로샷 분류 수행
labels_collected = []
scores_collected = []

for review in tqdm(reviews_to_predict):  # 리스트를 바로 순회
    output = classifier(review, candidate_labels, multi_label=True)
    labels = [label for label, score in zip(output['labels'], output['scores']) if score > 0.5]
    scores = [round(score, 2) for score in output['scores']]

    labels_collected.append(', '.join(labels))
    scores_collected.append(', '.join([f"{label}: {round(score, 2)}" for label, score in zip(output['labels'], output['scores'])]))

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame({
    'Review': reviews_to_predict,
    'Predicted_Labels': labels_collected,
    'Label_Scores': scores_collected
})

result_df


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
 67%|██████▋   | 10/15 [00:02<00:00,  5.83it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 15/15 [00:02<00:00,  5.20it/s]


,Review,Predicted_Labels,Label_Scores
0,맛은 있는데 양은 적어요,맛,"맛: 0.98, 가격: 0.34, 서비스: 0.33, 배달: 0.25, 양: 0.12"
1,가성비가 별로 안좋아요,가격,"가격: 0.93, 배달: 0.42, 양: 0.35, 서비스: 0.23, 맛: 0.01"
2,배 터질것 같아요,"양, 배달, 가격","양: 0.88, 배달: 0.59, 가격: 0.59, 서비스: 0.1, 맛: 0.08"
3,이렇게 늦을 거면 안 시켜 먹었죠,"서비스, 맛, 가격, 배달","서비스: 0.95, 맛: 0.75, 가격: 0.64, 배달: 0.52, 양: 0.25"
4,가격에 비해 양이 좀 적어요,"가격, 양","가격: 0.81, 양: 0.63, 서비스: 0.45, 배달: 0.33, 맛: 0.01"
5,맛있는데 너무 늦게 왔어요,"맛, 배달, 서비스","맛: 1.0, 배달: 0.98, 서비스: 0.88, 양: 0.39, 가격: 0.27"
6,맛에 비해 너무 비싸요,가격,"가격: 1.0, 배달: 0.19, 맛: 0.13, 양: 0.1, 서비스: 0.08"
7,다시는 안 시킬 것 같아요,서비스,"서비스: 0.64, 맛: 0.48, 가격: 0.48, 양: 0.46, 배달: 0.23"
8,또 시켜먹을게요,"서비스, 맛, 가격","서비스: 0.99, 맛: 0.94, 가격: 0.57, 양: 0.22, 배달: 0.12"
9,주문하고 30분 내로 왔어요,"서비스, 배달, 맛","서비스: 0.97, 배달: 0.75, 맛: 0.69, 가격: 0.41, 양: 0.27"


# 산술평균 예측(0.5이상 추가)

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from torch.utils.data import DataLoader, TensorDataset
import torch
from tqdm import tqdm
import pandas as pd

# 첫 번째 모델
# zeroshot_model30000 = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/프로젝트/final project/data/zero_model_epoch_2").to('cuda')
# model_zeroshot


# 두 번째 모델
model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
model = AutoModelForSequenceClassification.from_pretrained(model_name).to('cuda')
classifier = pipeline("zero-shot-classification", model=model, tokenizer=tokenizer, device=0)

# 공통 토크나이저
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# 데이터 전처리
inputs = tokenizer(reviews_to_predict, truncation=True, padding=True, return_tensors="pt").to('cuda')
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
data_loader = DataLoader(dataset, batch_size=16, shuffle=False)

# 모델 1 예측
scores_from_model1 = []

with torch.no_grad():
    for batch in tqdm(data_loader):
        input_ids, attention_mask = (item.to('cuda') for item in batch)
        outputs = model_zeroshot(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.sigmoid(logits).cpu().numpy()

        for prob in probs:
            scores = {candidate_labels[idx]: f"{val:.2f}" for idx, val in enumerate(prob)}
            scores_from_model1.append(scores)

# 모델 2 예측
scores_from_model2 = []

for review in tqdm(reviews_to_predict):
    output = classifier(review, candidate_labels, multi_label=True)
    scores = {label: round(score, 2) for label, score in zip(output['labels'], output['scores'])}
    scores_from_model2.append(scores)

averaged_scores_list = []
predicted_labels_list = []

for score1, score2 in zip(scores_from_model1, scores_from_model2):
    # 산술평균 저장
    averaged_scores = {}
    # 산술평균 점수가 0.5 이상인 레이블을 저장할 리스트
    predicted_labels = []

    for label in score1.keys():
        # 두 모델 점수 산술평균
        averaged_score = (float(score1[label]) + float(score2[label])) / 2
        # 산술평균 점수 소수점 둘째 자리 까지
        averaged_scores[label] = "{:.2f}".format(averaged_score)

        # 산술평균 점수가 0.5 이상이면 해당 레이블 추가
        if averaged_score > 0.5:
            predicted_labels.append(label)

    # 각 리뷰에 대한 산술평균 점수를 averaged_scores에 추가.
    averaged_scores_list.append(", ".join([f"{k}: {v}" for k, v in averaged_scores.items()]))
    # 각 리뷰에 대한 산술평균 점수가 0.5 이상인 레이블을 predicted_labels에 추가
    predicted_labels_list.append(", ".join(predicted_labels))

result_df = pd.DataFrame({
    'Review': reviews_to_predict,
    'Predicted_Labels': predicted_labels_list,
    'Averaged_Label_Scores': averaged_scores_list
})

result_df


 67%|██████▋   | 10/15 [00:01<00:00,  6.22it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 15/15 [00:02<00:00,  6.09it/s]


,Review,Predicted_Labels,Averaged_Label_Scores
0,맛은 있는데 양은 적어요,"맛, 양","맛: 0.99, 양: 0.55, 가격: 0.17, 배달: 0.12, 서비스: 0.17"
1,가성비가 별로 안좋아요,"맛, 가격","맛: 0.51, 양: 0.17, 가격: 0.97, 배달: 0.21, 서비스: 0.12"
2,배 터질것 같아요,맛,"맛: 0.54, 양: 0.44, 가격: 0.29, 배달: 0.29, 서비스: 0.05"
3,이렇게 늦을 거면 안 시켜 먹었죠,맛,"맛: 0.88, 양: 0.12, 가격: 0.32, 배달: 0.26, 서비스: 0.47"
4,가격에 비해 양이 좀 적어요,"양, 가격","맛: 0.13, 양: 0.81, 가격: 0.91, 배달: 0.17, 서비스: 0.23"
5,맛있는데 너무 늦게 왔어요,맛,"맛: 1.0, 양: 0.2, 가격: 0.14, 배달: 0.49, 서비스: 0.44"
6,맛에 비해 너무 비싸요,"맛, 가격","맛: 0.56, 양: 0.05, 가격: 1.0, 배달: 0.1, 서비스: 0.04"
7,다시는 안 시킬 것 같아요,맛,"맛: 0.74, 양: 0.23, 가격: 0.24, 배달: 0.12, 서비스: 0.32"
8,또 시켜먹을게요,맛,"맛: 0.97, 양: 0.11, 가격: 0.28, 배달: 0.06, 서비스: 0.49"
9,주문하고 30분 내로 왔어요,맛,"맛: 0.84, 양: 0.14, 가격: 0.2, 배달: 0.38, 서비스: 0.48"
